# Post-Processing
All parameters, which can/should be changed are described in the corresponding cell.

## Load Packages

In [ ]:
%matplotlib notebook
import util

## Load data

In [ ]:
# Initialize postprocessing
analyse = util.PostProcessor()
# Load data
# poisson_noise... spectra corrupted by poisson noise
analyse.load_data(poisson_noise = True)

## Align spectra (optional)

In [ ]:
# Select feature for cross correlation (sharp peak)
analyse.align_spectra_init()

In [ ]:
# Plot PCA-denoised and smoothed spectra, which are only used for aligning
s_denoised = analyse.plot_aligning(n=70, sigma = 2)

In [ ]:
# n... Number of components for reconstructing the spectra used for cross correlation
# vmin, vmax... contrast for spectra shift plot
analyse.align_spectra_calc(s_denoised, vmin = -0.5, vmax = 0.5)

In [ ]:
# max_shift.. maximum shift of the spectra
analyse.align_spectra_apply(s_denoised, max_shift = 0.2)

In [ ]:
#import matplotlib.pyplot as plt
#import os

# Location of saving results
#analyse.path_post = os.path.join(analyse.path_EELS, 'Post_Processing')

 # If folder is already present, ask to overwrite files
#if not os.path.exists(analyse.path_post):          
#    os.mkdir(analyse.path_post)
            
            
#fig = plt.gcf()  
#ax = plt.gca()  
#fig.set_size_inches(4, 3)
#ax.get_lines()[0].set_linewidth(4)
#ax.get_lines()[1].set_linewidth(2)
#ax.set_ylabel('Intensity / a.u.')
#ax.set_xlabel('Energy loss / eV')
#ax.set_title('')
#ax.axes.get_yaxis().set_ticks([])
#fig.tight_layout()
#fig.savefig(os.path.join(analyse.path_post, str('Aligning') + '.png'), dpi = 400,  bbox_inches='tight', pad_inches=0.01)
#print('Figure saved at: ' + os.path.join(analyse.path_post, str('Aligning') + '.png'))

In [ ]:
# Save current figure (latest open figure)
#analyse.save_fig(name = 'Spectra_Aligning')

## Crop spectra

In [ ]:
# Select cropping range for EELS
analyse.select_crop()

In [ ]:
# Crop EELS
analyse.crop()

## Clustering
algorithm from _'Dimensionality reduction and unsupervised clustering for EELS-SI'_

[https://doi.org/10.1016/j.ultramic.2021.113314](https://doi.org/10.1016/j.ultramic.2021.113314)

In [ ]:
# Initialize clustering (PCA decomposition)
# n_plot... maximum number of components in the scree plot
analyse.cluster_pca(n_plot = 60)

In [ ]:
# Dimension reduction with t-SNE
# n_denoise_cluster... number of components for denoising used for clustering-algorithm
# perplexity_tsne... t-SNE - related to the number of nearest neighbors that is used in other manifold learning algorithms (between 5 to 50)
analyse.clustering_init(n_denoise_cluster = 3, perplexity_tsne = 30)

In [ ]:
# OPTICS clustering
# eps_optics... estimate from reacability plot for clustering
#analyse.clustering(eps_optics = 30, cmap = 'hsv', shuffle = False) # --> better for continuous labeling
analyse.clustering(eps_optics = 90, cmap = 'tab10', shuffle = True) # --> better for discrete labeling and spectra averging

In [ ]:
# Save current figure (latest open figure)
#analyse.save_fig(name = 'Clustering')

In [ ]:
# Average spectra of same cluster
# k_min... minimum of points of the same cluster
analyse.clustering_spectra(k_min = 200)

## Find atom positions
using _Atomap_.

[https://doi.org/10.1186/s40679-017-0042-5](https://doi.org/10.1186/s40679-017-0042-5)

In [ ]:
# Determine atom positions
# s_low... minimum distance between atoms in pixel
analyse.atom_positioning(s_low = 2)

# Remove atoms
Selector = util.atom_selector(analyse.atom_positions, analyse.s_darkfield.data, map_label = False)

In [ ]:
# Add/Remove single atoms - close figure after finishing selecting
analyse.atom_positions = Selector.atom_positions
analyse.atom_positioning_single(analyse.s_darkfield.data, analyse.atom_positions, stacking = False)

In [ ]:
# Refine atom positions
analyse.refine_positions()

# Drift Correction

In [ ]:
# Calculate laticce vectors
analyse.lattice_calc()

In [ ]:
# Drift correction
# vector_ind... select index from lattice above, which should match the x_crystal and y_crystal vectors
# crystal_x... selected vector (vector_ind[0]) is shifted to crystal_x
# crystal_y... selected vector (vector_ind[1]) is shifted to crystal_y
analyse.drift_correction(vector_ind = [0, 1], crystal_x = [1,0], crystal_y = [0,1], scaling = False)

## Stacking & Aligning
Aligning by the _SmartAlign_ algorithm.

[https://doi.org/10.1186/s40679-015-0008-4](https://doi.org/10.1186/s40679-015-0008-4)

In [ ]:
# Choose atoms for stacking
# drift_corr... use drift corrected images or raw images
analyse.drift_on_off(drift_corr = True)
Selector = util.atom_selector(analyse.atom_position_stacking, analyse.darkfield_stacking, analyse.labels_shaped_stacking, analyse.newcmp, map_label = True)

In [ ]:
# Add/Remove single atoms for stacking - close figure after finishing selecting
analyse.atom_position_stacking = Selector.atom_positions
analyse.atom_positioning_single(analyse.darkfield_stacking, analyse.atom_position_stacking)

In [ ]:
# Crop cells and stack them
# width... width of the cells in pixels
# height... height of the cells in pixels
# shift_x... shifting all points in x-direction
# shift_y... shifting all points in y-direction
analyse.stacking(width = 45, height = 25, shift_x = -10, shift_y = 0)

In [ ]:
# Save current figure (latest open figure)
#analyse.save_fig(name = 'Stacking')

In [ ]:
# Alinging according SmartAlign algorithm
# i_rigid... Iterations for rigid alignment
# max_shift... Maximum rigid shift (finds maximum correlation within these limits)
# i_non_rigid_it... Iterations of non-rigid alignment (if blurry images, set to 0)
# i_non_rigid_max... Iteration of non-rigid alignment for each cell
# row... Lock distortion field ('fitted'... linear fit of distortion, 'locked'... constant fit of distortion field)

aligner = util.Aligner(analyse.dark_field_stack)

aligner.rigid_align(i_rigid = 1, max_shift = (4, 4))
aligner.non_rigid_align(i_non_rigid_it = 1, i_non_rigid_max = 500, row = 'locked')

aligner.plot_aligned()

## L2-norm

In [ ]:
# Calculate L2-norm
# norming... Norming function for each cell ('max'... divide by cell maximum, 'mean'... divide by cell mean, 'kernel'... gaussian smoothing)
# exponent... Using other norms as L2
analyse.darkfield_aligned = aligner.image_align
darkfield_aligned_norm = analyse.L2_init(norming = 'mean', exponent = 2)
util.L2_Selector(darkfield_aligned_norm, analyse.L2_norm)

In [ ]:
# Keep percentage of best slices
# n_ratio... Percentage of cells which should be kept (if None --> take calulated limit)
# norming... Use same norming function as above
# exponent... Use same exponent as above
analyse.L2_norm_process(n_ratio = 1, norming = 'mean', exponent = 2)

In [ ]:
# Save current figure (latest open figure)
#analyse.save_fig(name = 'Aligned')

In [ ]:
# Align EELS-signal
analyse.EELS_sum_aligned = aligner.align_second(analyse.EELS, analyse.index_image_stack, analyse.slice_L2_excluded)

## Fine structure mapping

In [ ]:
# Determine background and signal
analyse.EELS_region()

In [ ]:
# Plot background and signal
# background_fun... Background function ('PowerLaw' or 'Exponential')
# signal_plot... Plot only signal (otherwise: averaged, background, initial fit, fit)
analyse.EELS_background(background_fit = 'Powerlaw', signal_plot = False)

In [ ]:
# Interactive estimation of PCA-components
# background_fit... background fit function
# fast.. fast background removal
selector_pca = util.Selector_pca(analyse, background_fit = 'Power law', fast = True)

In [ ]:
# Save current figure (latest open figure)
#analyse.save_fig(name = 'PCA_denoising', size = (10,10))

In [ ]:
# Save denoised EELS spectrum with selected number of components for investigation in GMS (import rpl)
# pca_denoised... Save PCA-denoised spectrum image or averaged spectrum image
# n... Number of components used for denoising if pca_denoised is true (if none last selected number of components from plots is taken)
selector_pca.save_eels(pca_denoised = True, n = 3710)

## Save notebook for documentation

In [ ]:
%%javascript
var nb = IPython.notebook;
var kernel = IPython.notebook.kernel;
var command = "NOTEBOOK_FULL_PATH = '" + nb.base_url + nb.notebook_path + "'";
kernel.execute(command);

In [ ]:
util.saving_notebook(analyse.path_EELS, NOTEBOOK_FULL_PATH, name_notebook = '\\Post_processing.ipynb')